<a href="https://colab.research.google.com/github/Nrmnzcn/COVID19/blob/master/2_Covid19_Turkey_Compare_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install fbprophet

In [2]:
import numpy as np 
import pandas as pd 

from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from fbprophet.plot import add_changepoints_to_plot

import plotly.offline as py
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.metrics import mean_absolute_error

import itertools
from statsmodels.tsa.arima_model import ARIMA

from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.simplefilter('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tr-data/data.csv


In [3]:
column = ["Date", "Total Test", "Total Cases", "Total Deaths", "Total Recovered", 
          "Daily Cases", "Daily Test", "Daily Deaths", "Daily Recovered", "Active Cases" ]

In [4]:
data = pd.read_csv("../input/tr-data/data.csv")
data.Date = pd.to_datetime(data.Tarih)
data = data.fillna(0)
data.head()

,Tarih,Toplam Test,Toplam Vaka,Toplam Vefat,Toplam İyileşen,Günlük Hasta,Günlük Test,Günlük Vefat,Günlük İyileşen,Aktif Vaka
0,2021-04-05,39811595.0,3529601,32456.0,3130977.0,1706.0,245496.0,193.0,25627.0,366168.0
1,2021-04-04,39566099.0,3487050,32263.0,3105350.0,1508.0,246210.0,185.0,22674.0,349437.0
2,2021-04-03,39319889.0,3445052,32078.0,3082676.0,1483.0,249126.0,186.0,23214.0,330298.0
3,2021-04-02,39070763.0,3400296,31892.0,3059462.0,1471.0,248968.0,179.0,24419.0,308942.0
4,2021-04-01,38821795.0,3357988,31713.0,3035043.0,1424.0,243738.0,176.0,20817.0,291232.0


In [5]:
data.shape

(391, 10)

In [6]:
data.columns = column

case = data[["Date","Total Cases"]]
deaths = data[["Date","Total Deaths"]]
recoveries = data[["Date","Total Recovered"]]

In [7]:
case.columns = ['ds', 'y']
deaths.columns = ['ds', 'y']
recoveries.columns = ['ds', 'y']

In [8]:
f_date = pd.to_datetime('2020-03-11')
s_date = pd.to_datetime('2021-03-05')
e_date = pd.to_datetime('2021-04-05')
l_date = pd.to_datetime('2021-05-05')

In [9]:
# CASE

case_df = case.copy()
case_df.ds = pd.to_datetime(case_df.ds)

C_train = case_df.loc[(case_df['ds'] >= f_date) & (case_df['ds'] <= s_date)]
C_test = case_df.loc[(case_df['ds'] >= s_date) & (case_df['ds'] <= e_date)]
C_result = C_test.copy()

# DEATHS

deaths_df = deaths.copy()
deaths_df.ds = pd.to_datetime(deaths_df.ds)

D_train = deaths_df.loc[(deaths_df['ds'] >= f_date) & (deaths_df['ds'] <= s_date)]
D_test = deaths_df.loc[(deaths_df['ds'] >= s_date) & (deaths_df['ds'] <= e_date)]
D_result = D_test.copy()


# RECOVERIES

recoveries_df = recoveries.copy()
recoveries_df.ds = pd.to_datetime(recoveries_df.ds)

R_train = recoveries_df.loc[(recoveries_df['ds'] >= f_date) & (recoveries_df['ds'] <= s_date)]
R_test = recoveries_df.loc[(recoveries_df['ds'] >= s_date) & (recoveries_df['ds'] <= e_date)]
R_result = R_test.copy()

In [10]:
print(len(R_test))

32


In [11]:
Model_C = Prophet(changepoint_prior_scale=.5) 
Model_C .fit(C_train )

Model_D = Prophet(changepoint_prior_scale=.5) 
Model_D .fit(D_train )

Model_R = Prophet(changepoint_prior_scale=.5) 
Model_R .fit(R_train )

In [12]:
Future_C = Model_C.make_future_dataframe(periods=len(C_train))
Future_D = Model_D.make_future_dataframe(periods=len(D_train))
Future_R = Model_R.make_future_dataframe(periods=len(D_train))

Forecast_C = Model_C.predict(Future_C)
Forecast_D = Model_D.predict(Future_D)
Forecast_R = Model_R.predict(Future_R)

In [13]:
C_valitate = Forecast_C.loc[(Forecast_C['ds'] >= s_date) & (Forecast_C['ds'] <= e_date)]
C_result["PROPHET"] = C_valitate['yhat'].values

D_valitate = Forecast_D.loc[(Forecast_D['ds'] >= s_date) & (Forecast_D['ds'] <= e_date)]
D_result["PROPHET"] = D_valitate['yhat'].values

R_valitate = Forecast_R.loc[(Forecast_R['ds'] >= s_date) & (Forecast_R['ds'] <= e_date)]
R_result["PROPHET"] = R_valitate['yhat'].values

ARİMA


In [14]:
C_train = np.array(C_train.y, dtype=np.float64)
C_test = np.array(C_test.y, dtype=np.float64)

D_train = np.array(D_train.y, dtype=np.float64)
D_test = np.array(D_test.y, dtype=np.float64)

R_train = np.array(R_train.y, dtype=np.float64)
R_test = np.array(R_test.y, dtype=np.float64)

In [15]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(C_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(C_train, order=param)
result = model.fit()
C_prediction=result.forecast(steps=len(C_test))[0]
C_result['ARIMA'] = C_prediction

print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  358
Model:                 ARIMA(0, 2, 1)   Log Likelihood               -4464.375
Method:                       css-mle   S.D. of innovations          62515.309
Date:                Mon, 05 Apr 2021   AIC                           8934.751
Time:                        17:58:31   BIC                           8946.392
Sample:                             2   HQIC                          8939.380
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         59.3449     31.837      1.864      0.062      -3.055     121.745
ma.L1.D2.y    -1.0000      0.010   -103.676      0.000      -1.019      -0.981
                                    Roots           

In [16]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(D_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(D_train, order=param)
result = model.fit()
D_prediction=result.forecast(steps=len(C_test))[0]
D_result['ARIMA'] = D_prediction
print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  358
Model:                 ARIMA(3, 2, 3)   Log Likelihood               -1044.818
Method:                       css-mle   S.D. of innovations              4.451
Date:                Mon, 05 Apr 2021   AIC                           2105.637
Time:                        17:58:42   BIC                           2136.681
Sample:                             2   HQIC                          2117.983
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1362      0.746      0.183      0.855      -1.325       1.598
ar.L1.D2.y    -0.5753      0.041    -14.028      0.000      -0.656      -0.495
ar.L2.D2.y     0.4697      0.048      9.803      0.0

In [17]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(R_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(R_train, order=param)
result = model.fit()
R_prediction=result.forecast(steps=len(C_test))[0]
R_result['ARIMA'] = R_prediction
print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  358
Model:                 ARIMA(0, 2, 1)   Log Likelihood               -4444.202
Method:                       css-mle   S.D. of innovations          59090.030
Date:                Mon, 05 Apr 2021   AIC                           8894.404
Time:                        17:58:58   BIC                           8906.046
Sample:                             2   HQIC                          8899.034
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         58.7609     30.093      1.953      0.051      -0.220     117.742
ma.L1.D2.y    -1.0000      0.010   -100.874      0.000      -1.019      -0.981
                                    Roots           

**DECISION TREE REGRESSION**


In [18]:
def preprocessing(X, method):

    # scale, return[0-1]
    if method == 'MinMaxScaler':
        X_new = MinMaxScaler().fit_transform(X)

    return X_new

In [19]:
C_data = data[["Total Cases"]]
C_data = C_data.rename(columns={"Total Cases": "t"})

C_data["t-1"] = C_data["t"].shift(periods=1, fill_value=0)
C_data["t-2"] = C_data["t"].shift(periods=2, fill_value=0)
C_data["t-3"] = C_data["t"].shift(periods=3, fill_value=0)
C_data["t-4"] = C_data["t"].shift(periods=4, fill_value=0)
C_data["t-5"] = C_data["t"].shift(periods=5, fill_value=0)
C_data["t-6"] = C_data["t"].shift(periods=6, fill_value=0)
C_data["t-7"] = C_data["t"].shift(periods=7, fill_value=0)

C_data = C_data.replace([np.inf, -np.inf], np.nan)
C_data = C_data.dropna()

c_train = C_data.iloc[0:359]
c_test = C_data.iloc[359:]

print(len(c_test))

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_c_train = c_train[columns].values
y_c_train = c_train["t"].values

X_c_test = c_test[columns].values
y_c_test = c_test["t"].values


X_c_train = preprocessing(X_c_train, 'MinMaxScaler')
X_c_test = preprocessing(X_c_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_c_train, y_c_train)
pred = dt.predict(X_c_test)
C_result['DT'] = pred

32


In [20]:
D_data = data[["Total Deaths"]]
D_data = D_data.rename(columns={"Total Deaths": "t"})

D_data["t-1"] = D_data["t"].shift(periods=1, fill_value=0)
D_data["t-2"] = D_data["t"].shift(periods=2, fill_value=0)
D_data["t-3"] = D_data["t"].shift(periods=3, fill_value=0)
D_data["t-4"] = D_data["t"].shift(periods=4, fill_value=0)
D_data["t-5"] = D_data["t"].shift(periods=5, fill_value=0)
D_data["t-6"] = D_data["t"].shift(periods=6, fill_value=0)
D_data["t-7"] = D_data["t"].shift(periods=7, fill_value=0)

D_data = D_data.replace([np.inf, -np.inf], np.nan)
D_data = D_data.dropna()

d_train = D_data.iloc[0:359]
d_test = D_data.iloc[359:]

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_d_train = d_train[columns].values
y_d_train = d_train["t"].values

X_d_test = d_test[columns].values
y_d_test = d_test["t"].values


X_d_train = preprocessing(X_d_train, 'MinMaxScaler')
X_d_test = preprocessing(X_d_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_d_train, y_d_train)
pred = dt.predict(X_d_test)
D_result['DT'] = pred


In [21]:
R_data = data[["Total Recovered"]]
R_data = R_data.rename(columns={"Total Recovered": "t"})

R_data["t-1"] = R_data["t"].shift(periods=1, fill_value=0)
R_data["t-2"] = R_data["t"].shift(periods=2, fill_value=0)
R_data["t-3"] = R_data["t"].shift(periods=3, fill_value=0)
R_data["t-4"] = R_data["t"].shift(periods=4, fill_value=0)
R_data["t-5"] = R_data["t"].shift(periods=5, fill_value=0)
R_data["t-6"] = R_data["t"].shift(periods=6, fill_value=0)
R_data["t-7"] = R_data["t"].shift(periods=7, fill_value=0)

R_data = R_data.replace([np.inf, -np.inf], np.nan)
R_data = R_data.dropna()

r_train = R_data.iloc[0:359]
r_test = R_data.iloc[359:]

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_r_train = r_train[columns].values
y_r_train = r_train["t"].values

X_r_test = r_test[columns].values
y_r_test = r_test["t"].values


X_r_train = preprocessing(X_r_train, 'MinMaxScaler')
X_r_test = preprocessing(X_r_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_r_train, y_r_train)
pred = dt.predict(X_r_test)
R_result['DT'] = pred

**RESULT**


In [22]:
print(" ********** CASE **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((C_result['y'] - C_result['PROPHET']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((C_result['y'] - C_result['ARIMA']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((C_result['y'] - C_result['DT']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")


print(" ********** DEATHS **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((D_result['y'] - D_result['PROPHET']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((D_result['y'] - D_result['ARIMA']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((D_result['y'] - D_result['DT']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")

print(" ********** RECOVERIES **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((R_result['y'] - R_result['PROPHET']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((R_result['y'] - R_result['ARIMA']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((R_result['y'] - R_result['DT']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")

 ********** CASE **********
#### PROPHET ####
MAE :  274696.12053620024
MAPE :  8.583286914780636
#### ARIMA ####
MAE :  2994216.7146461643
MAPE :  97.9472195113021
#### DT ####
MAE :  1979541.8125
MAPE :  66.09814296888524


 ********** DEATHS **********
#### PROPHET ####
MAE :  2523.579742783993
MAPE :  8.480671692288212
#### ARIMA ####
MAE :  30225.22521415753
MAPE :  99.77126896480186
#### DT ####
MAE :  15025.71875
MAPE :  49.81129427038176


 ********** RECOVERIES **********
#### PROPHET ####
MAE :  209788.93922567097
MAPE :  7.2884348275126145
#### ARIMA ####
MAE :  2766168.634213751
MAPE :  97.62672141428497
#### DT ####
MAE :  1752471.625
MAPE :  61.57219384514


